In [ ]:
# Imports
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import mixed_precision
import tensorflow as tf

# Enable mixed precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Limit GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Dataset Overview
bleached_path = "./data/bleached_corals"
healthy_path = "./data/healthy_corals"

def get_image_paths(folder_path):
    return [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg')]

# Get all image paths
bleached_image_paths = get_image_paths(bleached_path)
healthy_image_paths = get_image_paths(healthy_path)

def load_and_resize_images(image_paths, size=(224, 224)):
    images = []
    for img_path in image_paths:
        with Image.open(img_path) as img:
            img = img.resize(size)
            images.append(np.array(img))
    return np.array(images)

bleached_images = load_and_resize_images(bleached_image_paths)
healthy_images = load_and_resize_images(healthy_image_paths)

# Labeling the dataset
bleached_labels = ['bleached'] * len(bleached_images)
healthy_labels = ['healthy'] * len(healthy_images)

data = np.vstack((bleached_images, healthy_images))
labels = np.array(bleached_labels + healthy_labels)

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split in training and testing data
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.3, random_state=42)

# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0

# Data Augmentation using tf.data
def preprocess_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    return image, label

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.batch(16).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(16).prefetch(tf.data.AUTOTUNE)

# Transfer Learning with InceptionV3
input_tensor = tf.keras.layers.Input(shape=(224, 224, 3))
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=input_tensor)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid', dtype='float32')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Unfreeze some layers of the base model
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Compile the Model with a different learning rate
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Early Stopping and Learning Rate Reduction Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Train the Model with Early Stopping and Data Augmentation
history = model.fit(train_dataset,
                    epochs=50,
                    validation_data=test_dataset,
                    callbacks=[early_stopping, reduce_lr])

# Evaluate the Model
loss, accuracy = model.evaluate(test_dataset)
print(f'Validation accuracy: {accuracy * 100:.2f}%')

# Confusion Matrix
y_pred = (model.predict(X_test) > 0.5).astype("int32")
cm = confusion_matrix(y_test, y_pred)

# Display Confusion Matrix using Seaborn
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

print("Classification Report:")
print(classification_report(y_test, y_pred))

# Plot training and validation accuracy for each epoch
train_accuracies = history.history['accuracy']
val_accuracies = history.history['val_accuracy']
epochs = range(1, len(train_accuracies) + 1)

plt.figure(figsize=(10, 7))
plt.plot(epochs, [acc * 100 for acc in train_accuracies], 'b', label='Training accuracy')
plt.plot(epochs, [acc * 100 for acc in val_accuracies], 'g--', label='Validation accuracy')  # Changed to green dashed line
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy per Epoch')
plt.legend()
plt.show()